The pretrained weights from SRGAN for the cassava dataset were initially obtained following the instructions available here: https://github.com/Lornatang/SRGAN-PyTorch.

# List of dependencies

In [ ]:
!pip install torch-summary
!pip install grad-cam

import torch
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Variable
from torchsummary import summary

# dataset
import os
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torchvision import transforms

# data augmentations
from torch.utils.data import DataLoader
from torch.utils.data.dataset import ConcatDataset

# model discriminator
import torch
import torch.nn as nn

# quantitative eval
from PIL import Image
from collections import Counter

# qualitative eval
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

import argparse
import cv2
import numpy as np
from torch.autograd import Function
from torchvision import models

# experiments
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

import copy

# Part A. Function Definitions

## 1. Dataset
Data loading and data split.

Source:

In [ ]:
class GetDataset(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, file_paths, labels, transform=None):
        'Initialization'
        self.imgs = [(img_path, label) for img_path, label in zip(file_paths, labels)]
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.file_paths)

    def __getitem__(self, index):
        'Generates one sample of data'

        # Select sample
        file_path = self.file_paths[index]
        label = self.labels[index]
        pil_image = Image.open(file_path)

        # Check if image has only single channel. If True, then swap with 0th image
        # Assumption 0th image has got 3 number of channels
        if len(pil_image.getbands()) != 3:
            file_path = self.file_paths[0]
            label = self.labels[0]
            pil_image = Image.open(file_path)

        # Convert image to torch tensor
        if self.transform != None:
            tr_image = self.transform(pil_image)
        else:
            tr_image = transforms.ToTensor()(pil_image).unsqueeze_(0)

        return tr_image, label
    

def split_train_into_train_val(train_file_ids, train_file_paths, train_labels, test_size=0.1):
    """
    Split train_file_paths and train_labels to train_file_paths, val_file_paths and
    train_labels, val_labels
    """

    # Create a mapping between image_id and file_path
    image_id_name_map = dict(zip(train_file_ids, train_file_paths))

    # Get validation files and validation labels separate
    train_file_ids, val_file_ids, train_labels, val_labels = train_test_split(
        train_file_ids, train_labels, test_size=test_size, random_state=5, shuffle=True
    )
    train_file_paths = [image_id_name_map[image_id] for image_id in train_file_ids]
    val_file_paths = [image_id_name_map[image_id] for image_id in val_file_ids]

    print ("Length of train files list", len(train_file_paths))
    print ("Length of train labels", len(train_labels))
    print ("Length of val files list", len(val_file_paths))
    print ("Length of val labels", len(val_labels))

    return train_file_ids, val_file_ids, train_file_paths, val_file_paths, train_labels, val_labels


def get_train_test_file_paths_n_labels():
    """
    Get array train_file_paths, train_labels, test_file_paths and test_labels
    """

    # Data loading and data discriminators set up
    images_data_dir = r'../input/SRGANPyTorch_Lornatang/data/TRAIN'
    train_test_split_file = r'../input/SRGANPyTorch_Lornatang/from-rg/train_test_split.txt'
    images_file = r'../input/SRGANPyTorch_Lornatang/from-rg/images.txt'
    labels_file = r'../input/SRGANPyTorch_Lornatang/from-rg/image_class_labels.txt'

    # Read the images_file which stores image-id and image-name mapping
    image_file_id_df = pd.read_csv(images_file, sep=' ', header=None)
    image_file_id_mat = image_file_id_df.values
    image_id_name_map = dict(zip(image_file_id_mat[:, 0], image_file_id_mat[:, 1]))

    # Read the train_test_split file which stores image-id and train-test split mapping
    image_id_train_test_split_df = pd.read_csv(train_test_split_file, sep=' ', header=None)
    image_id_train_test_split_mat = image_id_train_test_split_df.values
    image_id_train_test_split_map = dict(zip(image_id_train_test_split_mat[:, 0],
                                             image_id_train_test_split_mat[:, 1]))

    # Read the image class labels file
    image_id_label_df = pd.read_csv(labels_file, sep=' ', header=None)
    image_id_label_mat = image_id_label_df.values
    image_id_label_map = dict(zip(image_id_label_mat[:, 0], image_id_label_mat[:, 1]))

    # Put together train_files, train_labels, test_files and test_labels lists
    train_image_ids, test_image_ids = [], []
    train_file_paths, test_file_paths = [], []
    train_labels, test_labels = [], []
    for file_id in image_id_name_map.keys():
        file_name = image_id_name_map[file_id]
        is_train = image_id_train_test_split_map[file_id]
        label = image_id_label_map[file_id] - 1  # To ensure labels start from 0

        if is_train:
            train_image_ids.append(file_id)
            train_file_paths.append(os.path.join(images_data_dir, file_name))
            train_labels.append(label)
        else:
            test_image_ids.append(file_id)
            test_file_paths.append(os.path.join(images_data_dir, file_name))
            test_labels.append(label)

    print ("Length of test files list", len(test_file_paths))
    print ("Length of test labels list", len(test_labels))

    return train_image_ids, test_image_ids, train_file_paths, test_file_paths, train_labels, test_labels

## 2. SRGAN Discriminator Model
Source: https://github.com/Lornatang/SRGAN-PyTorch

In [ ]:
# Copyright 2020 Dakewe Biotech Corporation. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

class Discriminator(nn.Module):
    r"""The main architecture of the discriminator. Similar to VGG structure."""

    def __init__(self):
        super(Discriminator, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # input is (3) x 96 x 96
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=False),  # state size. (64) x 48 x 48
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=False),  # state size. (128) x 24 x 24
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=False),  # state size. (256) x 12 x 12
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1, bias=False),  # state size. (512) x 6 x 6
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 6 * 6, 1024),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        out = self.features(input)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out


def discriminator() -> Discriminator:
    r"""GAN model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1609.04802>`_ paper.
    """
    model = Discriminator()
    return model

## 3. Model Evaluation - Quantitative
### 3.1 Calculate loss, accuracy, confusion matrix, precision, recall, and F1-score
Source:

Adjustments: 
* Added class weights to cross-entropy loss function to address class imbalance
* Added confusion matrix, precision, recall, F1, macro-F1, and weighted F1-scores to be printed after every epoch
* Return self.network from the train function to be used for GradCAM visualization

In [ ]:
def get_count_correct_preds(network_output, target):

    output = network_output
    pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
    pred.data = pred.data.view_as(target.data)
    correct = target.eq(pred).sum().item()

    return correct

class ModelTrainTest():

    def __init__(self, network, device, model_file_path, threshold=1e-4):
        super(ModelTrainTest, self).__init__()
        self.network = network
        self.device = device
        self.model_file_path = model_file_path
        self.threshold = threshold
        self.train_loss = 1e9
        self.val_loss = 1e9

    def train(self, optimizer, epoch, params_max_norm, train_data_loader, val_data_loader):
        self.network.train()
        train_loss = 0
        correct = 0
        cnt_batches = 0
        
        #added class weights
        class_weights = torch.FloatTensor([466/5656, 1443/5656, 773/5656, 2658/5656, 316/5656]) #cbb, cbsd, cgm, cmd, healthy
        
        for batch_idx, (data, target) in enumerate(train_data_loader):
            data, target = Variable(data).to(self.device), Variable(target).to(self.device)

            optimizer.zero_grad()
            output = self.network(data) 
            
            criterion_weighted = nn.CrossEntropyLoss(weight=class_weights).cuda()
            loss = criterion_weighted(output, target)
            loss.backward()

            clip_grad_norm_(self.network.parameters(), params_max_norm)
            optimizer.step()

            correct += get_count_correct_preds(output, target)
            train_loss += loss.item()
            cnt_batches += 1

            del data, target, output

        train_loss /= cnt_batches
        val_loss, val_acc = self.test(epoch, val_data_loader)

        if val_loss < self.val_loss - self.threshold:
            self.val_loss = val_loss
            torch.save(self.network.state_dict(), self.model_file_path)

        train_acc = correct / len(train_data_loader.dataset)

        print('\nAfter epoch {} - Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            epoch, train_loss, correct, len(train_data_loader.dataset),
            100. * correct / len(train_data_loader.dataset)))

        return train_loss, train_acc, val_loss, val_acc, self.network #added self.network for gradCAM visualization

    def test(self, epoch, test_data_loader):
        self.network.eval()
        test_loss = 0
        correct = 0

        nb_classes = 5

        confusion_matrix = torch.zeros(nb_classes, nb_classes)


        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_data_loader): 
                data, target = Variable(data).to(self.device), Variable(target).to(self.device)
                output = self.network(data)
                test_loss += F.nll_loss(output, target, size_average=False).item()  # sum up batch loss

                correct += get_count_correct_preds(output, target)

                ##added cf, PR, f1
                preds = output.data.max(1, keepdim=True)[1]
                for t, p in zip(target.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1

                del data, target, output
        
        print(confusion_matrix)
        print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
        cm = confusion_matrix.cpu().data.numpy()
        recall = np.diag(cm) / np.sum(cm, axis = 1)
        precision = np.diag(cm) / np.sum(cm, axis = 0)
        print('recall', recall)
        print('precision', precision)

        f1 = 2 * precision * recall / (precision + recall)
        f1 = np.nan_to_num(f1)
        macro_f1 = np.sum(f1)/nb_classes
        weighted_f1 = np.sum(np.multiply(f1, np.sum(cm, axis=1)))/np.sum(cm)
        print('f1_score', f1)
        print('macro_f1', macro_f1)
        print('weighted_f1', weighted_f1)
        
        test_loss /= len(test_data_loader.dataset)
        test_acc = correct / len(test_data_loader.dataset)
        print('\nAfter epoch {} - Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            epoch, test_loss, correct, len(test_data_loader.dataset),
            100. * correct / len(test_data_loader.dataset)))

        return  test_loss, test_acc


### 3.2 Plot loss and accuracy

In [ ]:
def plot_downstream_loss(train_losses, val_losses, experiment_name):
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.title('Loss plots')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()
    plt.savefig('classification_loss_{}.jpg'.format(experiment_name))

def plot_downstream_accuracy(train_accs, val_accs, experiment_name):
    plt.plot(train_accs)
    plt.plot(val_accs)
    plt.ylim(0.2,1.0)
    plt.title('Accuracy plots')
    plt.ylabel('Accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()
    plt.savefig('classification_acc_{}.jpg'.format(experiment_name))

### 3.3 Save results in csv

In [ ]:
def save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name):
    pd.set_option('display.max_rows', None)

    results_df = pd.DataFrame()
    results_df['epoch count'] = [i for i in range(1, epochs + 1)]
    results_df['train loss'] = train_losses
    results_df['val loss'] = val_losses
    results_df['train acc'] = train_accs
    results_df['val acc'] = val_accs

    results_file_path = '{}_observations.csv'.format(experiment_name)
    
    results_df.to_csv(results_file_path)
    display(results_df)

## 4. Model Evaluation - Qualitative
Grad-CAM Visualization. 
Source: https://github.com/yaleCat/Grad-CAM-pytorch

In [ ]:
class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """

    def __init__(self, model, blob_name, target_layers):
        self.model = model
        self.blob_name = blob_name
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for idx, module in self.model._modules.items():
            if idx != self.blob_name:
                try:
                    x = module(x)
                except:
                    x = x.view(x.size(0), -1)
                    x = module(x)
            else:
                for name, block in enumerate(getattr(self.model,self.blob_name)):
                    x = block(x)
                    if str(name) in self.target_layers:
                        x.register_hook(self.save_gradient)
                        outputs += [x]    
        return outputs, x

def preprocess_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[:, :, ::-1]
    for i in range(3):
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
    preprocessed_img = torch.from_numpy(preprocessed_img)
    preprocessed_img.unsqueeze_(0)
    inputs = preprocessed_img.requires_grad_(True)
    return inputs

def show_cams(img, mask_dic, experiment_name):
    for name, mask in mask_dic.items():
        show_cam_on_image(img, mask, name, experiment_name)
    
def show_cam_on_image(img, mask, name, experiment_name):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    cv2.imwrite("cam{}_{}.jpg".format(name, experiment_name), np.uint8(255 * cam))
    plt.imshow(np.uint8(255 * cam)[:,:,::-1])
    plt.show()

class GradCam:
    def __init__(self, model, blob_name, target_layer_names, use_cuda):
        self.model = model
        self.target_layer_names = target_layer_names
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.extractor = FeatureExtractor(self.model, blob_name, target_layer_names)

    def __call__(self, inputs, index=None):
        cam_dic = {}
        if self.cuda:
            features, output = self.extractor(inputs.cuda())
        else:
            features, output = self.extractor(inputs)

        if index == None:
            index = np.argmax(output.cpu().data.numpy())
        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)
        self.model.zero_grad()
        one_hot.backward()
        self.model.zero_grad()
        for idx, feature in enumerate(features):
            grads_val = self.extractor.gradients[len(features)-1-idx].cpu().data.numpy()
            target = features[idx]
            target = target.cpu().data.numpy()[0, :]
            weights = np.mean(grads_val, axis=(2, 3))[0, :]
            cam = np.zeros(target.shape[1:], dtype=np.float32)
            for i, w in enumerate(weights):
                cam += w * target[i, :, :]
            cam = np.maximum(cam, 0)
            cam = cv2.resize(cam, (88, 88))
            cam = cam - np.min(cam)
            cam = cam / np.max(cam)
            cam_dic[self.target_layer_names[idx]] = cam
        return cam_dic


class GuidedBackpropReLU(Function):

    @staticmethod
    def forward(self, i):
        positive_mask = (i > 0).type_as(i)
        output = torch.addcmul(torch.zeros(i.size()).type_as(i), i, positive_mask)
        self.save_for_backward(i)
        return output

    @staticmethod
    def backward(self, grad_output):
        i = self.saved_tensors[0]
        grad_input = None
        positive_mask_1 = (i > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(i.size()).type_as(i),
                                   torch.addcmul(torch.zeros(i.size()).type_as(i), grad_output,
                                                 positive_mask_1), positive_mask_2)

        return grad_input

class GuidedBackpropSwish(Function):

    @staticmethod
    def forward(self, i):
        result = i * torch.sigmoid(i)
        self.save_for_backward(i)
        return result

    @staticmethod
    def backward(self, grad_output):
        i = self.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        positive_mask_1 = (i > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i))) * positive_mask_1 * positive_mask_2
        return grad_input
        
class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda, activation_layer_name = 'ReLU'):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        if activation_layer_name == 'MemoryEfficientSwish':
            fb_func = GuidedBackpropSwish.apply
        else:
            fb_func = GuidedBackpropReLU.apply
        for idx0, module0 in self.model._modules.items():
            module0 = self.model._modules[idx0]
            if module0.__class__.__name__ == activation_layer_name:
                self.model._modules[idx0] = fb_func
            for idx1, _ in module0._modules.items():
                module1 = module0._modules[idx1]
                if module1.__class__.__name__ == activation_layer_name:
                    self.model._modules[idx0]._modules[idx1] = fb_func
                    continue
                for idx2, _ in module1._modules.items():
                    module2 = module1._modules[idx2]
                    if module2.__class__.__name__ == activation_layer_name:
                        self.model._modules[idx0]._modules[idx1]._modules[idx2] = fb_func
                    
    def forward(self, inputs):
        return self.model(inputs)

    def __call__(self, inputs, index=None):
        if self.cuda:
            output = self.forward(inputs.cuda())
        else:
            output = self.forward(inputs)

        if index == None:
            index = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)
        one_hot.backward()
        gradient = inputs.grad.cpu().data.numpy()
        gradient = gradient[0, :, :, :]
        return gradient

def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)

def show_gbs(inputs, gb_model, target_index, mask_dic, experiment_name):
    gb = gb_model(inputs, index=target_index)
    gb = gb.transpose((1, 2, 0))
    for idx, mask in mask_dic.items():
        cam_mask = cv2.merge([mask, mask, mask])
        cam_gb = deprocess_image(cam_mask*gb)
        cv2.imwrite('cam_gb{}_{}.jpg'.format(idx, experiment_name), cam_gb)
    cv2.imwrite('gb_{}.jpg'.format(experiment_name), deprocess_image(gb))
    
    plt.subplot(1,2,1)
    plt.imshow(cam_gb)
    plt.subplot(1,2,2)
    plt.imshow(deprocess_image(gb))
    plt.show()

# Part B. Implementation

## 1. Dataset
### 1.1 Data split

In [ ]:
# Data loading and data discriminators set up
train_image_ids, test_image_ids, train_file_paths, test_file_paths, train_labels, test_labels = \
    get_train_test_file_paths_n_labels()

# Get validation files and validation labels separate
train_image_ids, val_image_ids, train_file_paths, val_file_paths, train_labels, val_labels = \
    split_train_into_train_val(train_image_ids, train_file_paths, train_labels, test_size=0.1)

### 1.2 Data augmentations
Resize, center crop, horizontal flip, darkness and lightness jitter, rotation

In [ ]:
# Define data loaders
batch_size = 32

def_data_transform = transforms.Compose([
    transforms.CenterCrop((88, 88)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

hflip_data_transform = transforms.Compose([
    transforms.CenterCrop((88, 88)),
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

darkness_jitter_transform = transforms.Compose([
    transforms.CenterCrop((88, 88)),
    transforms.ColorJitter(brightness=[0.5, 0.9]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

lightness_jitter_transform = transforms.Compose([
    transforms.CenterCrop((88, 88)),
    transforms.ColorJitter(brightness=[1.1, 1.5]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

rotations_transform = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.CenterCrop((88, 88)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

all_in_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=[0.5, 1.5]),
    transforms.RandomRotation(degrees=15),
    transforms.CenterCrop((88, 88)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [ ]:
train_data_gen = ConcatDataset(
        [GetDataset(train_file_paths, train_labels, def_data_transform),
         GetDataset(train_file_paths, train_labels, hflip_data_transform),
         GetDataset(train_file_paths, train_labels, darkness_jitter_transform),
         GetDataset(train_file_paths, train_labels, lightness_jitter_transform),
         GetDataset(train_file_paths, train_labels, rotations_transform),
         GetDataset(train_file_paths, train_labels, all_in_transform)])
train_data_loader = DataLoader(train_data_gen, batch_size = batch_size, shuffle = True, num_workers = 8)

val_data_gen = GetDataset(val_file_paths, val_labels, def_data_transform)
val_data_loader = DataLoader(val_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

test_data_gen = GetDataset(test_file_paths, test_labels, def_data_transform)
test_data_loader = DataLoader(test_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

## 2. Model
### 2.1 Load and freeze weights from pretrained network

In [ ]:
model = Discriminator()

In [ ]:
# load pretrained weights
checkpoint = torch.load('../input/srgansrresnetdiscriminator032221/SRGAN_0322.pth')
model.load_state_dict(checkpoint)

In [ ]:
# examine the architecture
model.eval()

In [ ]:
# freeze weights from pretrained network
for param in model.parameters():
    param.requires_grad = False

### 2.2 Change classifier for downstream classification
Sigmoid is changed to logsoftmax; number of output classes is changed to 5

In [ ]:
num_filters = model.classifier[2].in_features
num_classes = 5

model.classifier[2] = nn.Linear(num_filters, num_classes)

model.classifier[3] = nn.LogSoftmax()

## 3. Experiments
The experiments we've run involve image resizing, adjusting the depth of the architecture, and adding dropouts.

### Trial 1: Original SRGAN discriminator
Run the original model to create a baseline.

In [ ]:
experiment_name = 'NoResize_Centercrop88'

# Set device on which training is done plus optimizer to use
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

#deepcopy model
model_to_train = copy.deepcopy(model) 
#move model to cpu; retain model_to_train in gpu
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL1 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL1, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested index.
    target_index = None
    
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 2: Resize 256; center crop 88

In [ ]:
resize = transforms.Resize((256, 256))
centercrop = transforms.CenterCrop((88, 88))

def_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

hflip_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

darkness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[0.5, 0.9]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

lightness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[1.1, 1.5]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

rotations_transform = transforms.Compose([
    resize,
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

all_in_transform = transforms.Compose([
    resize,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=[0.5, 1.5]),
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_data_gen = ConcatDataset(
        [GetDataset(train_file_paths, train_labels, def_data_transform),
         GetDataset(train_file_paths, train_labels, hflip_data_transform),
         GetDataset(train_file_paths, train_labels, darkness_jitter_transform),
         GetDataset(train_file_paths, train_labels, lightness_jitter_transform),
         GetDataset(train_file_paths, train_labels, rotations_transform),
         GetDataset(train_file_paths, train_labels, all_in_transform)])

train_data_loader = DataLoader(train_data_gen, batch_size = batch_size, shuffle = True, num_workers = 8)

val_data_gen = GetDataset(val_file_paths, val_labels, def_data_transform)
val_data_loader = DataLoader(val_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

test_data_gen = GetDataset(test_file_paths, test_labels, def_data_transform)
test_data_loader = DataLoader(test_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

In [ ]:
experiment_name = 'Resize256_Centercrop88'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL2 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL2, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 3: Resize 128; center crop 88

In [ ]:
resize = transforms.Resize((128, 128))
centercrop = transforms.CenterCrop((88, 88))

def_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

hflip_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

darkness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[0.5, 0.9]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

lightness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[1.1, 1.5]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

rotations_transform = transforms.Compose([
    resize,
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

all_in_transform = transforms.Compose([
    resize,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=[0.5, 1.5]),
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_data_gen = ConcatDataset(
        [GetDataset(train_file_paths, train_labels, def_data_transform),
         GetDataset(train_file_paths, train_labels, hflip_data_transform),
         GetDataset(train_file_paths, train_labels, darkness_jitter_transform),
         GetDataset(train_file_paths, train_labels, lightness_jitter_transform),
         GetDataset(train_file_paths, train_labels, rotations_transform),
         GetDataset(train_file_paths, train_labels, all_in_transform)])

train_data_loader = DataLoader(train_data_gen, batch_size = batch_size, shuffle = True, num_workers = 8)

val_data_gen = GetDataset(val_file_paths, val_labels, def_data_transform)
val_data_loader = DataLoader(val_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

test_data_gen = GetDataset(test_file_paths, test_labels, def_data_transform)
test_data_loader = DataLoader(test_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

In [ ]:
experiment_name = 'Resize128_Centercrop88'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL3 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL3, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 4: Resize 88; no center crop

In [ ]:
resize = transforms.Resize((88, 88))
centercrop = None

def_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

hflip_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

darkness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[0.5, 0.9]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

lightness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[1.1, 1.5]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

rotations_transform = transforms.Compose([
    resize,
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

all_in_transform = transforms.Compose([
    resize,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=[0.5, 1.5]),
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_data_gen = ConcatDataset(
        [GetDataset(train_file_paths, train_labels, def_data_transform),
         GetDataset(train_file_paths, train_labels, hflip_data_transform),
         GetDataset(train_file_paths, train_labels, darkness_jitter_transform),
         GetDataset(train_file_paths, train_labels, lightness_jitter_transform),
         GetDataset(train_file_paths, train_labels, rotations_transform),
         GetDataset(train_file_paths, train_labels, all_in_transform)])

train_data_loader = DataLoader(train_data_gen, batch_size = batch_size, shuffle = True, num_workers = 8)

val_data_gen = GetDataset(val_file_paths, val_labels, def_data_transform)
val_data_loader = DataLoader(val_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

test_data_gen = GetDataset(test_file_paths, test_labels, def_data_transform)
test_data_loader = DataLoader(test_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

In [ ]:
experiment_name = 'Resize88_NoCentercrop'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL4 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)


In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL4, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 5: Resize 128; crop 88; remove last convolution block

In [ ]:
resize = transforms.Resize((128, 128))
centercrop = transforms.CenterCrop((88, 88))

def_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

hflip_data_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

darkness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[0.5, 0.9]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

lightness_jitter_transform = transforms.Compose([
    resize,
    centercrop,
    transforms.ColorJitter(brightness=[1.1, 1.5]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

rotations_transform = transforms.Compose([
    resize,
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

all_in_transform = transforms.Compose([
    resize,
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=[0.5, 1.5]),
    transforms.RandomRotation(degrees=15),
    centercrop,
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_data_gen = ConcatDataset(
        [GetDataset(train_file_paths, train_labels, def_data_transform),
         GetDataset(train_file_paths, train_labels, hflip_data_transform),
         GetDataset(train_file_paths, train_labels, darkness_jitter_transform),
         GetDataset(train_file_paths, train_labels, lightness_jitter_transform),
         GetDataset(train_file_paths, train_labels, rotations_transform),
         GetDataset(train_file_paths, train_labels, all_in_transform)])

train_data_loader = DataLoader(train_data_gen, batch_size = batch_size, shuffle = True, num_workers = 8)

val_data_gen = GetDataset(val_file_paths, val_labels, def_data_transform)
val_data_loader = DataLoader(val_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

test_data_gen = GetDataset(test_file_paths, test_labels, def_data_transform)
test_data_loader = DataLoader(test_data_gen, batch_size=batch_size, shuffle=True, num_workers=8)

In [ ]:
experiment_name = 'Resize128_Centercrop88_Remove1ConvBlock'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

In [ ]:
# remove last features block (each block consists of Conv2d-BatchNorm2d-LeakyReLU)
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])

# adjust input size to the initial layer of the classifier
model_to_train.classifier[0] = nn.Linear(in_features=61952, out_features=1024, bias=True)

# inspect the architecture
model_to_train.eval()

In [ ]:
# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj2 = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL5 = model_train_test_obj2.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL5, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 6: Resize 128; crop 88; remove last 2 convolution blocks

In [ ]:
experiment_name = 'Resize128_Centercrop88_Remove2ConvBlocks'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

model_to_train.to(device)
sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

In [ ]:
# remove last 2 features block (each block consists of Conv2d-BatchNorm2d-LeakyReLU)
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])

# adjust input size to the initial layer of the classifier
model_to_train.classifier[0] = nn.Linear(in_features=30976, out_features=1024, bias=True)

# inspect the architecture
model_to_train.eval()

In [ ]:
# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj2 = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL6 = model_train_test_obj2.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL6, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 7: Resize 128; crop 88; remove last 3 convolution blocks

In [ ]:
experiment_name = 'Resize128_Centercrop88_Remove3ConvBlocks'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

model_to_train.to(device)
sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

In [ ]:
# remove last 3 features block (each block consists of Conv2d-BatchNorm2d-LeakyReLU)
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])

# adjust input size to the initial layer of the classifier
model_to_train.classifier[0] = nn.Linear(in_features=256*22*22, out_features=1024, bias=True)

# inspect the architecture
model_to_train.eval()

In [ ]:
# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj2 = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL7 = model_train_test_obj2.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL7, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 8: Resize 128; center crop 88; remove last conv block; dropout p=0.5

In [ ]:
experiment_name = 'Resize128_Centercrop88_Dropout05'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

In [ ]:
# remove last features block (each block consists of Conv2d-BatchNorm2d-LeakyReLU)
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])

# adjust input size to the initial layer of the classifier
model_to_train.classifier[0] = nn.Linear(in_features=61952, out_features=1024, bias=True)

# add dropout after leaky ReLU
model.classifier[1] = nn.Sequential(nn.LeakyReLU(negative_slope=0.2, inplace=True),
                                   nn.Dropout(p=0.5,inplace=False))

# inspect the architecture
model_to_train.eval()

In [ ]:
# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL9 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL9, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 10: Resize 128; center crop 88; remove last conv block; dropout p=0.7

In [ ]:
experiment_name = 'Resize128_Centercrop88_Dropout07'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

In [ ]:
# remove last features block (each block consists of Conv2d-BatchNorm2d-LeakyReLU)
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])

# adjust input size to the initial layer of the classifier
model_to_train.classifier[0] = nn.Linear(in_features=61952, out_features=1024, bias=True)

# add dropout after leaky ReLU
model.classifier[1] = nn.Sequential(nn.LeakyReLU(negative_slope=0.2, inplace=True),
                                   nn.Dropout(p=0.7,inplace=False))

# inspect the architecture
model_to_train.eval()

In [ ]:
# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL10 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL10, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))

### Trial 11: Resize 128; center crop 88; remove last conv block; dropout p=0.9

In [ ]:
experiment_name = 'Resize128_Centercrop88_Dropout09'

epochs = 45
lr = 1e-4
weight_decay_const = 5e-4
optim = 'adam'

model.to('cuda:0')
model_to_train = copy.deepcopy(model)
model.to('cpu')

sgd_optimizer = optim.SGD(model_to_train.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay_const)
adam_optimizer = optim.Adam(model_to_train.parameters(), lr=lr, weight_decay=weight_decay_const)

if optim == 'sgd':
    optimizer = sgd_optimizer
else:
    optimizer = adam_optimizer

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True, min_lr=1e-5)

In [ ]:
# remove last features block (each block consists of Conv2d-BatchNorm2d-LeakyReLU)
model_to_train.features = nn.Sequential(*list(model_to_train.features.children())[:-3])

# adjust input size to the initial layer of the classifier
model_to_train.classifier[0] = nn.Linear(in_features=61952, out_features=1024, bias=True)

# add dropout after leaky ReLU
model.classifier[1] = nn.Sequential(nn.LeakyReLU(negative_slope=0.2, inplace=True),
                                   nn.Dropout(p=0.9,inplace=False))

# inspect the architecture
model_to_train.eval()

In [ ]:
# start training
model_file_path = 'cassava_downstream_discriminator_{}.pth'.format(experiment_name)
model_train_test_obj = ModelTrainTest(model_to_train, device, model_file_path)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

print('Training started')
for epoch_no in range(epochs):
    train_loss, train_acc, val_loss, val_acc, TRAINED_MODEL10 = model_train_test_obj.train(
        optimizer=optimizer, epoch=epoch_no, params_max_norm=4,
        train_data_loader=train_data_loader, val_data_loader=val_data_loader
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    scheduler.step(val_loss)

In [ ]:
save_display_results(epochs, train_losses, val_losses, train_accs, val_accs, experiment_name)

In [ ]:
plot_downstream_loss(train_losses, val_losses, experiment_name)

In [ ]:
plot_downstream_accuracy(train_accs, val_accs, experiment_name)

In [ ]:
grad_cam = GradCam(model=TRAINED_MODEL11, blob_name = 'features', target_layer_names=['20'], use_cuda=True)

for i in ['train-cbb-68', 'train-cbsd-177', 'train-cmd-180', 'train-healthy-67', 'train-cgm-651']:
    print(i)
    img = cv2.imread('../input/cassavagradcam/cassava-gradcam/{}.jpg'.format(i), 1)
    img = np.float32(cv2.resize(img, (88, 88))) / 255
    inputs = preprocess_image(img)

    target_index = None
    mask_dic = grad_cam(inputs, target_index)
    show_cams(img, mask_dic, '{}_{}'.format(i, experiment_name))
    gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU', use_cuda=True)
    show_gbs(inputs, gb_model, target_index, mask_dic, '{}_{}'.format(i, experiment_name))